<div>
<img src="https://discuss.pytorch.org/uploads/default/original/2X/3/35226d9fbc661ced1c5d17e374638389178c3176.png" width="400" style="margin: 50px auto; display: block; position: relative; left: -30px;" />
</div>

<!--NAVIGATION-->
# < [Optimization](3-Optimization.ipynb) | Modules | [CNN](5-CNN.ipynb) >

### PyTorch Modules

Modules are a way to build re-usable model components and to manage model parameters.
PyTorch has many built-in modules for common operations like convolutions, recurrent neural networks, max pooling, common activation functions, etc.
You can also build your own modules.

This notenook introduces modules, and you will build a small neural network to perform image classification on MNIST.

### Table of Contents
#### 1. [Modules](#Modules)
#### 2. [Building and Training a Neural Network](#Building-and-Training-a-Neural-Network)

---

# Modules

### Modules manage parameters

They help to
- keep track of the parameters in your model.
- save/load of your model.
- reset gradients (with `model.zero_grad()`)
- move all parameters to the gpu (with `model.cuda()`)

The model parameter's are represented by `torch.nn.Parameter` objects.
A `Parameter` is a tensor with `requires_grad` set to `True` by default, and which is automatically added to the list of parameters when used within a model. If you are interested, you can have a look at the [torch.nn.Parameter documentation](https://pytorch.org/docs/stable/_modules/torch/nn/parameter.html).

As an example, a `Conv1d` module has two parameters: `weight` as `bias`:

In [0]:
import torch
import matplotlib.pyplot as plt

In [2]:
module = torch.nn.Conv1d(5, 2, 3)
print(module.weight)

Parameter containing:
tensor([[[-0.1558,  0.1440, -0.2207],
         [ 0.0481,  0.1372,  0.0622],
         [ 0.1867, -0.0912, -0.1304],
         [-0.2502,  0.1512, -0.1497],
         [-0.2282, -0.2377,  0.0279]],

        [[ 0.1316,  0.2579,  0.2547],
         [ 0.2069, -0.2146, -0.0430],
         [-0.1838, -0.2480,  0.1647],
         [-0.1884,  0.1834,  0.1668],
         [-0.1321, -0.0003, -0.1440]]], requires_grad=True)


---

## Basic usage

Each instance of a model has its own __parameters__.
The parameters are initialized randomly when the model is instantiated.

In [3]:
linear_regression_model = torch.nn.Linear(5, 2)  # 5 input dimensions, 2 output dimensions

linear_regression_model.weight  # running this cell more than once, you'll see different outputs

Parameter containing:
tensor([[ 0.2448, -0.1887, -0.3816,  0.0501, -0.2270],
        [-0.1158, -0.3796,  0.1035,  0.3248, -0.3272]], requires_grad=True)

Models operate on __batches__ of data. If a model is designed to operate on datapoints with 5 features, the shape of the model's inputs will be `(batch, 5)`. This allows us to process multiple datapoints in parallel and increase efficiency.

In [4]:
batch_size = 3
x = torch.randn(batch_size, 5)
y = torch.randn(batch_size, 2)

print("x = {}\ny = {}".format(x, y))

x = tensor([[-0.3001,  2.4366,  1.5277, -0.5393, -0.5692],
        [-0.9455,  0.6452, -0.8166, -1.6170,  1.3204],
        [ 0.7241, -0.0366,  0.1380, -2.4934, -0.5083]])
y = tensor([[-0.5250,  1.7239],
        [-1.8024, -0.0761],
        [ 1.0230, -1.2565]])


You can __call__ the model on an input (forward pass). This evaluation uses the current model parameters.

In [5]:
predicted_y = linear_regression_model(x)
print("predicted y = {}".format(predicted_y))

predicted y = tensor([[-1.2876, -1.1031],
        [-0.6957, -1.5593],
        [-0.1515, -1.0813]], grad_fn=<AddmmBackward>)


To optimize the model, you compute a __measure of error__ (loss) on the predictions. In this case, we use a squared error:

In [6]:
# Compute a loss
loss = torch.sum((y - predicted_y)**2)
loss

tensor(13.4087, grad_fn=<SumBackward0>)

With the loss computed, we compute __gradients__ of the loss with respect to all model parameters using automatic differentiation.

In [7]:
linear_regression_model.zero_grad()  # clear previous gradient
loss.backward()

print("\nGradients:")
print(linear_regression_model.weight.grad)
print(linear_regression_model.bias.grad)


Gradients:
tensor([[ -3.3358,  -2.2027,  -4.4621,   3.1003,   4.9849],
        [  4.7552, -15.7037,  -6.1672,   6.9725,  -0.8767]])
tensor([-1.6609, -8.2701])


You can now use these gradients to optimize the model parameters.

---

## Composing modules with `torch.nn.Sequential`

If the model you want to build is a simple chain of other modules, you can compose them using `torch.nn.Sequential`:

In [8]:
neural_net = torch.nn.Sequential(
    torch.nn.Linear(5, 10),
    torch.nn.ReLU(),
    torch.nn.Linear(10, 2),
)

# Run the model:
neural_net(x)

tensor([[ 0.2345,  0.1927],
        [ 0.3738, -0.1847],
        [ 0.4717,  0.2260]], grad_fn=<AddmmBackward>)

---

## Module parameters

You can inspect your network's parameters using `.parameters()` or `.named_parameters()`:

In [9]:
for param, tensor in neural_net.named_parameters():
    print("{:10s} shape = {}".format(param, tensor.shape))

0.weight   shape = torch.Size([10, 5])
0.bias     shape = torch.Size([10])
2.weight   shape = torch.Size([2, 10])
2.bias     shape = torch.Size([2])


---

## Custom modules

A module has to implement two functions:

- the `__init__` function, where you define all the sub-components that have learnable parameters. This makes sure that your module becomes aware all its parameters. The sub-components (layers) do not need to be defined in order of execution or connceted together. Don't forget to initialize the parent class `torch.nn.Module` with `super().__init__()`.


- the `forward` function, which is the method that defines what has to be executed during the forward pass and especially how the layers are connected. This is where you call the layers that you defined inside `__init__`.


In [0]:
# This is the most basic form of a custom module:
class MySuperSimpleModule(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__() # initialise from parent class
        
        # Define sub-modules or parameters
        # torch.nn.Module takes care of adding their parameters to your new module
        self.linear = torch.nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = self.linear(x)
        return out

You can use the `print` function to list a model's submodules and parameters:

In [11]:
model = MySuperSimpleModule(input_size=20, num_classes=5)
print(model)

MySuperSimpleModule(
  (linear): Linear(in_features=20, out_features=5, bias=True)
)


You can use `model.parameters()` to get the list of parameters of your model automatically inferred by PyTorch.

In [12]:
for name, p in model.named_parameters():  # Here we use a sligtly different version of the parameters() function
    print(name, ":\n", p)                 # which also returns the parameter name

linear.weight :
 Parameter containing:
tensor([[ 0.1550, -0.0019,  0.1778,  0.2124, -0.0644,  0.0588, -0.0475,  0.1749,
          0.0418,  0.1175,  0.1130, -0.0289, -0.0716, -0.2224, -0.0476, -0.1832,
          0.0538,  0.1858, -0.0181, -0.0951],
        [-0.0784, -0.0137, -0.1376,  0.0189, -0.1963, -0.1175, -0.1321, -0.2008,
         -0.1476,  0.1345, -0.1782,  0.0968,  0.1965,  0.2138, -0.1965, -0.0122,
         -0.1692,  0.0206, -0.0355, -0.1863],
        [-0.1806, -0.0510, -0.0015, -0.1116,  0.0608,  0.0171,  0.1257,  0.0306,
         -0.0862,  0.1175,  0.1457,  0.0622, -0.0889,  0.1515, -0.0072,  0.0048,
          0.0891, -0.1616, -0.0555,  0.0903],
        [-0.0283, -0.1980, -0.0944, -0.0455, -0.0536, -0.1514,  0.1236, -0.1304,
          0.1520,  0.0404,  0.0145,  0.0433, -0.2147, -0.1342,  0.0269,  0.0527,
          0.1948,  0.0953,  0.0609,  0.1652],
        [-0.1493,  0.1712, -0.0271,  0.1650,  0.1918, -0.0936, -0.0582,  0.1113,
         -0.1037,  0.0447,  0.0044,  0.0980,  0.

---

# Building and Training a Neural Network

It's time to implement a neural network now. In this section, you will learn to classify handwritten digits from the widely known MNIST dataset.
The dataset consists of 60,000 training images of size 28x28, and another 10,000 images for evaluating the quality of the trained model.

![MNIST](https://github.com/theevann/amld-pytorch-workshop/blob/master/figures/mnist.jpeg?raw=1)

## Loading the dataset

MNIST is widely used and a dataset and it is available in the `torchvision` library.

In [13]:
import torchvision

# MNIST Dataset (Images and Labels)
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True
)
test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    transform=torchvision.transforms.ToTensor()
)

  0%|          | 16384/9912422 [00:00<01:14, 133034.96it/s]

9920512it [00:00, 30273161.54it/s]                           


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


32768it [00:00, 439419.37it/s]
  1%|          | 16384/1648877 [00:00<00:11, 143235.40it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


1654784it [00:00, 7516905.62it/s]                            
8192it [00:00, 185166.81it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


A dataset in PyTorch behalves _like an array_. It has a length, and you can access its entries with `dataset[i]`.

__Exercise__<br>
Verify how many images there are in the training dataset. How is one training example represented? What is the type and shape of an entry from the dataset?

In [17]:
n_images = len(train_dataset)
print(f"There are {n_images} images in the train dataset.")

first_datapoint = train_dataset[0]
print(f"Data points are a {type(first_datapoint)} of length {len(first_datapoint)}.")
print("The entries are the input and the correct class (the digit.)")

X, y = first_datapoint
print(f"The images are {type(X)}'s of shape {X.shape}.")

There are 60000 images in the train dataset.
Data points are a <class 'tuple'> of length 2.
The entries are the input and the correct class (the digit.)
The images are <class 'torch.Tensor'>'s of shape torch.Size([1, 28, 28]).


When we train a model, we make multiple passes through all the examples in the training set. Each pass, the data points are shuffled and batched together. For this purpose, we use a `DataLoader`. The `DataLoader` support multi-threading to optize your data-loading pipeline.

In [0]:
# Dataset Loader (Input Batcher)
batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

We can now iterate through the dataset in shuffled batches. Everytime you do this, the order will be different.

In [0]:
for images, labels in train_loader:
    assert len(images) == batch_size
    assert len(labels) == batch_size

__Exercise__<br>
Search in the documentation how to enable multi-threading in the data loaders.

In [0]:
### Multi-threading in dataloaders

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=1
)

## Defining the model

A fully-connected neural network consists of layers that contain a number of values (neurons) computed as linear combinations of the neurons in the layer before. The first layer contains the network's input (your features), and the last layer contains the prediction. In our case, the last layer contains 10 neurons that are trained to be large when an input image is of the corresponding digit (0, 1, 2, 3, 4, 5, 6, 7, 8, 9).

The parameters of this model that are optimized (trained), are the weights that connect the neurons. These are drawn as edges in the illustration below.

![](https://raw.githubusercontent.com/ledell/sldm4-h2o/master/mlp_network.png)

To make sure that neural networks can approximate non-linear functions, each neuron's value is transformed with some non-linear transformation function $\sigma(\cdot)$, often called an ‘activation function’ before being fed as input to the next layer. 

To be precise, the neurons $\vec x_{i+1}$ in layer $i+1$ are computed from the neurons $\vec x_i$ in layer $i$ as 

$$ \vec x_{i+1} = \sigma\left(W_{i+1} \vec x_i + \vec b_{i+1} \right) $$

where $W_{i+1}$ encodes the network parameters between each pair of input/output neurons in layer $i+1$, and $\vec b_{i+1}$ contains 'bias terms'. $\sigma$ operates element-wise.

A layer like that can be implemented using `torch.nn.Linear` followed by an activation function such as `torch.nn.ReLU` or `torch.nn.Sigmoid`.

---

__Exercise__<br>
Implement a multi-layer fully-connected neural network with two hidden layers and the following numbers of neurons in each layer:

- Input-size: *input_size*
- 1st hidden layer: 75
- 2nd hidden layer: 50
- Output layer: *num_classes*

Use `ReLU`s as ‘activation functions’ in between each pair of layers, but not after the last layer.

In [0]:
import torch.nn.functional as F  # provides some helper functions like Relu's, Sigmoids, Tanh, etc.

class MyNeuralNetwork(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(MyNeuralNetwork, self).__init__()
        
        self.input_size = input_size
        self.num_classes = num_classes
        
        self.linear_1 = torch.nn.Linear(input_size, 75)
        self.linear_2 = torch.nn.Linear(75,50)
        self.linear_3 = torch.nn.Linear(50, num_classes)
        
    
    def forward(self, x):
        out = F.relu(self.linear_1(x))
        out = F.relu(self.linear_2(out))
        out = self.linear_3(out)
        return out

Now feed an input to your network:

In [22]:
x = torch.rand(16, 28 * 28)  # the first dimension is reserved for the 'batch_size'
model = MyNeuralNetwork(input_size=28 * 28, num_classes=10)
out = model(x)  # this calls model.forward(x)
out[0, :]

tensor([ 0.1579, -0.1346, -0.0773,  0.0448,  0.0337, -0.0154, -0.0247,  0.0481,
         0.0048,  0.1203], grad_fn=<SliceBackward>)

__Exercise__ <br>
What does `out[0, :]` above represent?

---

## Training the model

Most of the functions to train a model follow a similar pattern in PyTorch.
In most of the cases in consists of the following steps:
- Loop over data (in batches)
- Create a prediction (forward pass)
- Clear previous gradients (!)
- Compute gradients (backward pass)
- Parameter update (using an optimizer)

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Copy all model parameters to the GPU
model = model.to(device)

# Define the Loss function and Optimizer that you want to use
criterion = torch.nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)  # NOTE: model.parameters()

# Passes over the whole dataset
for epoch in range(5):
    total_loss = 0.0
    
    # Loop over batches in the training set
    for (inputs, labels) in train_loader:
        
        # Move inputs from CPU memory to GPU memory
        inputs = inputs.to(device)
        labels = labels.to(device)

        # The pixels in our images have a square 28x28 structure, but the network
        # accepts a *vector* of inputs. We therefore reshape it.
        # -1 is a special number that indicates 'whatever is left'
        inputs = inputs.view(-1, 28*28)

        # Do a forward pass, loss computation, compute gradient, and optimize
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Add up training losses so we can compute an average later
        total_loss += loss.item()
        
    print("Epoch %d, Loss=%.4f" % (epoch+1, total_loss/len(train_loader)))

Epoch 1, Loss=0.4280
Epoch 2, Loss=0.1765
Epoch 3, Loss=0.1303
Epoch 4, Loss=0.1028
Epoch 5, Loss=0.0841


Note:
- We can use the `.to` function on the model directly. Indeed, since PyTorch knows all the model parameters, it can put all the parameters on the correct device.
- We use `model.parameters()` to get all the parameters of the model and we can instantiate an optimizer that will optimize these parameters `torch.optim.SGD(model.parameters())`.
- To apply the forward function of the module, we write `model(input)`. In most cases, `model.forward(inputs)` would also work, but there is a slight difference : PyTorch allows you to register hook functions for a model that are automatically called when you do a forward pass on your model. Using `model(input)` will call these hooks and then call the forward function, while using `model.forward(inputs)` will just silently ignore them.

Do you see the convenience of Modules?

### Loss functions

PyTorch comes with a lot of predefined loss functions :
- `L1Loss`
- `MSELoss`
- `CrossEntropyLoss`
- `NLLLoss`
- `PoissonNLLLoss`
- `KLDivLoss`
- `BCELoss`
- `...`

Check out the [PyTorch Documentation](https://pytorch.org/docs/master/nn.html#loss-functions).

### Assessing model performance 
This function loops over another `data_loader` (usually containing test/validation data) and computes the model's accuracy on it.

In [0]:
def accuracy(model, data_loader, device):
    with torch.no_grad(): # during model evaluation, we don't need the autograd mechanism (speeds things up)
        correct = 0
        total = 0
        for inputs, labels in data_loader:
            inputs = inputs.to(device)     
            inputs = inputs.view(-1, 28*28)
            
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            
            correct += (predicted.cpu() == labels).sum().item()
            total += labels.size(0)
            
    acc = correct / total
    return acc

In [25]:
accuracy(model, test_loader, device)  # look at: accuracy(model, train_loader, device)

0.9674

### We get an accuracy of around 97%. Can you improve this?

---

## Storing and loading models

In [26]:
torch.save(model, "my_model.pt")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type MyNeuralNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
my_model_loaded = torch.load("my_model.pt")

In [0]:
print(model.linear_3.bias)
print(my_model_loaded.linear_3.bias)

___

This intro to modules used [this medium post](https://medium.com/deeplearningbrasilia/deep-learning-introduction-to-pytorch-5bd39421c84) as a resource.

<!--NAVIGATION-->
# < [Optimization](3-Optimization.ipynb) | Modules | [CNN](5-CNN.ipynb) >